# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_api_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_api_df

,Unnamed: 0,lat,lon,max_temp,humidity,wind_speed,cloudiness,country,date,name
0,0,-34.7333,135.8667,62.87,54,15.37,100,AU,1665364935,Port Lincoln
1,1,4.7717,33.5903,73.13,72,2.93,100,SS,1665364935,Kapoeta
2,2,5.8933,95.3214,80.69,77,10.58,100,ID,1665364936,Sabang
3,3,-46.1927,168.8643,58.23,60,5.37,100,NZ,1665364936,Mataura
4,4,60.7333,77.5889,36.73,93,7.11,100,RU,1665364936,Strezhevoy
...,...,...,...,...,...,...,...,...,...,...
526,526,19.9103,109.6859,72.43,79,14.27,90,CN,1665365104,Lingao
527,527,21.6422,69.6093,81.79,80,4.85,35,IN,1665365105,Porbandar
528,528,2.0167,112.9333,76.64,90,1.21,100,MY,1665365105,Kapit
529,529,50.0979,118.0369,37.26,38,5.28,13,RU,1665365106,Krasnokamensk


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# Store latitude and longitude in locations
locations = weather_api_df[["lat", "lon"]]

# Store Humidity in humidity
humidity = weather_api_df["humidity"]

In [17]:
mean_lat = weather_api_df['lat'].mean()
mean_lon = weather_api_df['lon'].mean()

print(mean_lat)
print(mean_lon)

20.78483653483995
22.1532551789077


In [21]:
# Plot Heatmap
fig = gmaps.figure(center=(20.78, 22.15), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

mean_lat = sum_stats.iloc[1,0]
mean_lng = sum_stats.iloc[1,1]### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down the DataFrame to find your ideal weather condition.
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

cleaned_weather_df = weather_api_df.loc[(weather_api_df['max_temp'] >= 70) & (weather_api_df['max_temp'] <= 80) & \
                                        (weather_api_df['wind_speed'] < 10) & (weather_api_df['cloudiness'] == 0)]
                                        
                                        
cleaned_weather_df.dropna() 

,Unnamed: 0,lat,lon,max_temp,humidity,wind_speed,cloudiness,country,date,name
78,78,-17.7125,-39.2481,75.34,82,6.55,0,BR,1665364959,Caravelas
149,149,9.4092,45.0640,72.30,59,3.87,0,SO,1665364979,Oodweyne
151,151,-19.8436,34.8389,70.43,91,3.33,0,MZ,1665364980,Beira
179,179,-21.9500,27.9000,70.75,28,5.01,0,BW,1665364992,Tobane
223,223,36.3703,30.2869,71.19,53,5.46,0,TR,1665365008,Kumluca
366,366,-16.8392,36.9856,70.05,91,2.89,0,MZ,1665365056,Mocuba
458,458,24.4667,85.6000,74.88,86,4.97,0,IN,1665365083,Kodarmā
479,479,-12.9274,34.2961,72.52,45,3.91,0,MW,1665365090,Nkhotakota


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# Create a hotel_df with city name, country, lat, lon
hotel_df = cleaned_weather_df.loc[:,["name","country", "lat", "lon"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,name,country,lat,lon,Hotel Name
78,Caravelas,BR,-17.7125,-39.2481,
149,Oodweyne,SO,9.4092,45.0640,
151,Beira,MZ,-19.8436,34.8389,
179,Tobane,BW,-21.9500,27.9000,
223,Kumluca,TR,36.3703,30.2869,
366,Mocuba,MZ,-16.8392,36.9856,
458,Kodarmā,IN,24.4667,85.6000,
479,Nkhotakota,MW,-12.9274,34.2961,


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [33]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))